# Dowloading Deep Learning Youtube Video Comments

# 0. Import Libraries and Set Output Filename

In [1]:

import pandas as pd
import json
import os
import sys
import re
import time

import requests

# pandas dataframe display configuration
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/comment_service_ajax'

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
# csv file name
FILE_NAME = 'ytb_comments.csv'

# set parameters
# filter comments by popularity or recent, 0:False, 1:True
SORT_BY_POPULAR = 0
# default recent False, change to 1 to download latest comments
SORT_BY_RECENT = 0
# set comment limit
COMMENT_LIMIT = 100

YT_CFG_RE = r'ytcfg\.set\s*\(\s*({.+?})\s*\)\s*;'
YT_INITIAL_DATA_RE = r'(?:window\s*\[\s*["\']ytInitialData["\']\s*\]|ytInitialData)\s*=\s*({.+?})\s*;\s*(?:var\s+meta|</script|\n)'

# 1. Set up some helper functions: ajax_request, download_comments etc.

In [2]:
def regex_search(text, pattern, group=1, default=None):
    match = re.search(pattern, text)
    return match.group(group) if match else default


def ajax_request(session, endpoint, ytcfg, retries=5, sleep=20):
    url = 'https://www.youtube.com' + endpoint['commandMetadata']['webCommandMetadata']['apiUrl']
    
    data = {'context': ytcfg['INNERTUBE_CONTEXT'],
            'continuation': endpoint['continuationCommand']['token']}

    for _ in range(retries):
        response = session.post(url, params={'key': ytcfg['INNERTUBE_API_KEY']}, json=data)
        if response.status_code == 200:
            return response.json()
        if response.status_code in [403, 413]:
            return {}
        else:
            time.sleep(sleep)

def download_comments(YOUTUBE_VIDEO_URL, sort_by=SORT_BY_RECENT, language=None, sleep=0.1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT
    response = session.get(YOUTUBE_VIDEO_URL)

    if 'uxe=' in response.request.url:
        session.cookies.set('CONSENT', 'YES+cb', domain='.youtube.com')
        response = session.get(YOUTUBE_VIDEO_URL)

    html = response.text
    ytcfg = json.loads(regex_search(html, YT_CFG_RE, default=''))
    if not ytcfg:
        return # Unable to extract configuration
    if language:
        ytcfg['INNERTUBE_CONTEXT']['client']['hl'] = language

    data = json.loads(regex_search(html, YT_INITIAL_DATA_RE, default=''))

    section = next(search_dict(data, 'itemSectionRenderer'), None)
    renderer = next(search_dict(section, 'continuationItemRenderer'), None) if section else None
    if not renderer:
        # Comments disabled?
        return

    needs_sorting = sort_by != SORT_BY_POPULAR
    continuations = [renderer['continuationEndpoint']]
    while continuations:
        continuation = continuations.pop()
        response = ajax_request(session, continuation, ytcfg)

        if not response:
            break
        if list(search_dict(response, 'externalErrorMessage')):
            raise RuntimeError('Error returned from server: ' + next(search_dict(response, 'externalErrorMessage')))

        if needs_sorting:
            sort_menu = next(search_dict(response, 'sortFilterSubMenuRenderer'), {}).get('subMenuItems', [])
            if sort_by < len(sort_menu):
                continuations = [sort_menu[sort_by]['serviceEndpoint']]
                needs_sorting = False
                continue
            raise RuntimeError('Failed to set sorting')

        actions = list(search_dict(response, 'reloadContinuationItemsCommand')) + \
                  list(search_dict(response, 'appendContinuationItemsAction'))
        for action in actions:
            for item in action.get('continuationItems', []):
                if action['targetId'] == 'comments-section':
                    # Process continuations for comments and replies.
                    continuations[:0] = [ep for ep in search_dict(item, 'continuationEndpoint')]
                if action['targetId'].startswith('comment-replies-item') and 'continuationItemRenderer' in item:
                    # Process the 'Show more replies' button
                    continuations.append(next(search_dict(item, 'buttonRenderer'))['command'])

        for comment in reversed(list(search_dict(response, 'commentRenderer'))):
            yield {'cid': comment['commentId'],
                   'text': ''.join([c['text'] for c in comment['contentText'].get('runs', [])]),
                   'time': comment['publishedTimeText']['runs'][0]['text'],
                   'author': comment.get('authorText', {}).get('simpleText', ''),
                   'channel': comment['authorEndpoint']['browseEndpoint'].get('browseId', ''),
                   'votes': comment.get('voteCount', {}).get('simpleText', '0'),
                   'photo': comment['authorThumbnail']['thumbnails'][-1]['url'],
                   'heart': next(search_dict(comment, 'isHearted'), False)}

        time.sleep(sleep)

def search_dict(partial, search_key):
    stack = [partial]
    while stack:
        current_item = stack.pop()
        if isinstance(current_item, dict):
            for key, value in current_item.items():
                if key == search_key:
                    yield value
                else:
                    stack.append(value)
        elif isinstance(current_item, list):
            for value in current_item:
                stack.append(value)

# output function

In [3]:
def main(url):
    """
    This function will save the comments into a dataframe and output a csv file
    By default, it will append the comments in csv, not overwriting them, change it in line 34
    To preview the comments in json, uncomment the lines - 22 and 23
    """
    df_comment = pd.DataFrame()
    try:
        youtube_url = url
        limit = COMMENT_LIMIT

        print('Downloading Youtube comments for video:', youtube_url)

        count = 0

        start_time = time.time()

        for comment in download_comments(youtube_url):

            df_comment = df_comment.append(comment, ignore_index=True)

            count += 1

            if limit and count >= limit:
                break

        print("DataFrame Shape: ",df_comment.shape,"\nComment DataFrame: ")
        display(df_comment)

        if not os.path.isfile(FILE_NAME):
            df_comment.to_csv(FILE_NAME, encoding='utf-8', index=False)
        else:  # else it exists so append without writing the header
            df_comment.to_csv(FILE_NAME, mode='a', encoding='utf-8', index=False, header=False)

        print('\n[{:.2f} seconds] Done!'.format(time.time() - start_time))

    except Exception as e:
        print('Error:', str(e))
        sys.exit(1)

# 2. Saving Comments 

## Link 1: LSTM is dead. Long Live Transformers!

In [16]:
youtube_URL = 'https://www.youtube.com/watch?v=S27pHKBEp30&ab_channel=SeattleAppliedDeepLearning'
main(youtube_URL)

DataFrame Shape:  (20, 8) 
Comment DataFrame: 


,cid,text,time,author,channel,votes,photo,heart
0,UgxgkEfJOBfumbsyVEx4AaABAg,You folks need to look into asymptotics and Pa...,2 jaar geleden (bewerkt),Bijou Smith,UCQHd4WVQV3pXdEzJlqbKLYg,2,https://yt3.ggpht.com/ytc/AMLnZu9wS5hqpG4vTGN5...,0.0
1,UgwYrxBBfpcAMuJXIKp4AaABAg,That's one of the best deep learning related p...,2 jaar geleden,Fernando Marcos Wittmann,UCSkGKRwgugM9PvjVq2yEr6w,283,https://yt3.ggpht.com/ytc/AMLnZu_r4wddwcpUfkHM...,0.0
2,UgzK2Ob_pB8pRQ1bu-54AaABAg,Great talk. It's always thrilling to see someo...,2 jaar geleden,Monika Thornton,UCu5k0NV1R20wqbouAfyt8pQ,35,https://yt3.ggpht.com/ytc/AMLnZu_-kKcTHWtQOa_x...,0.0
3,Ugzvb7tGZvM4Rn6tS3d4AaABAg,Thank you for this concise and well-rounded ta...,2 jaar geleden,Richard Osuala,UCJ-6cO-Wbp9UUnNZNODvUug,39,https://yt3.ggpht.com/ytc/AMLnZu-pazOcWKS8CLqs...,0.0
4,UgxWDOKVlylna0agtdh4AaABAg,World deserve more lectures like this one. I d...,3 weken geleden,Bartosz Bielecki,UCxz26sGKakQDQa49ZvE9Rbw,0,https://yt3.ggpht.com/ytc/AMLnZu-SklrReJm8d9Bj...,0.0
5,Ugx0CdqI8JvoYVg1Kbp4AaABAg,Leo is an excellent professor. He explains dif...,1 jaar geleden,Jagdeep Sandhu,UC3Pr96zPp3nUqLh2flhFoWg,9,https://yt3.ggpht.com/ytc/AMLnZu_gKqZJJUKZ3fyi...,0.0
6,UgxrZz7IrvXfVSVBUiV4AaABAg,12:56 the review of the pseudocode of the atte...,2 jaar geleden,Scranny,UC7sXi9wl_Xln351GU1s8czQ,13,https://yt3.ggpht.com/ytc/AMLnZu-n6a_LjNeg4tev...,0.0
7,UgzhO9iXyXAM28QKm5F4AaABAg,Its hard to overstate just how much this topic...,2 jaar geleden (bewerkt),Ajit Kirpekar,UC4vvvcc0v-17XiFmVE8Jcbg,3,https://yt3.ggpht.com/ytc/AMLnZu91CAyuXd5eND6S...,0.0
8,UgwS29XuWABJnXwuGWp4AaABAg,Very nice recap of Transformers and what sets ...,2 jaar geleden,David Teschner,UCCTltHfVCRZoYzd9Y_88zdw,5,https://yt3.ggpht.com/ytc/AMLnZu_mmodh5yMXGURz...,0.0
9,UgxT8wTASkTjOBRQ58N4AaABAg,I was trying to use similar super-low frequenc...,2 jaar geleden (bewerkt),Nobody Quite,UC9P48v2dU72YUK4NGpN5AwA,9,https://yt3.ggpht.com/ytc/AMLnZu9WxMZcLUs9EdyF...,0.0



[0.98 seconds] Done!


In [29]:
df_comment.text.head(10)

0                                  Shot horizontally😀👍
1    I have a feeling we aren't too far from Disney...
2    That’s incredible. I can’t believe how much wo...
3    I love stuff like this because I feel like art...
4    That…was…EPIC! Seeing that live would have bee...
5    So awesome. Imagine when companies start doing...
6    Absolutely mind blowing especially at the begi...
7    The crowd reactions are awesome. Makes you thi...
8    John Williams's contribution to cinema is one ...
9    That's amazing! Just imaging what they'll be a...
Name: text, dtype: object

In [18]:
df_comment1 = pd.read_csv('./ytb_comments.csv')
df_comment1.head()

,cid,text,time,author,channel,votes,photo,heart
0,UgyQFWUAwflZw8dgkUl4AaABAg,Shot horizontally😀👍,1 jaar geleden,Dave Ellis,UC2vxO8ZCkPanvHF5L4IQ79Q,340,https://yt3.ggpht.com/ytc/AMLnZu_80GFCyeIQRDI_...,1.0
1,UgwtAC15ZpDsqRFscuB4AaABAg,I have a feeling we aren't too far from Disney...,11 maanden geleden,Stefan,UCn_cd_kyCntJs4yu249FayQ,259,https://yt3.ggpht.com/ytc/AMLnZu9dxMjVLZVsbgtJ...,1.0
2,Ugxl-2CmqdAsscaV9FJ4AaABAg,That’s incredible. I can’t believe how much wo...,11 maanden geleden,Tim Moeller,UCLjV7s8SwS3STCvWCZ_Znqw,177,https://yt3.ggpht.com/qA5LGkIhqbY3mRa0PEM01uui...,1.0
3,Ugxjl5CzAJ928JZ6pdt4AaABAg,I love stuff like this because I feel like art...,1 jaar geleden,Jack Middleton,UCHNIRmyVDh52gm60BDEzyCw,126,https://yt3.ggpht.com/ytc/AMLnZu92mQwpJfkOX2Qh...,1.0
4,UgxSGTgzQNP0B0e7OPh4AaABAg,That…was…EPIC! Seeing that live would have bee...,11 maanden geleden,Isaac Sheppard,UC-Kkc303MEP_1heObyPye8Q,78,https://yt3.ggpht.com/ytc/AMLnZu8nVB2_P5WkAx-_...,1.0


## Comments from A List of Deep Learning Courses

In [21]:
ytb_video_list = ['https://www.youtube.com/watch?v=Mubj_fqiAv8&list=PLeo1K3hjS3uu7CxAacxVndI4bE_o3BDtO&ab_channel=codebasics',
                  'https://www.youtube.com/watch?v=YFNKnUhm_-s&list=PLZoTAELRMXVPGU70ZGsckrMdr0FteeRUi&ab_channel=KrishNaik',
                  'https://www.youtube.com/watch?v=7sB052Pz0sQ&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI&ab_channel=AlexanderAmini']

for video_link in ytb_video_list:
    main(video_link)

DataFrame Shape:  (19, 8) 
Comment DataFrame: 


,cid,text,time,author,channel,votes,photo,heart
0,UgwF5s8UP709OoBaziJ4AaABAg,Do you want to learn technology from me? Check...,6 maanden geleden (bewerkt),@codebasics,UCh9nVJoWXmFb7sLApWGcLPQ,3,https://yt3.ggpht.com/Gfb034LSTAvgWjlKObV-QauI...,0.0
1,UgzDWjn25mqmzkd1a8V4AaABAg,The amount of love and respect for you Sir is ...,2 jaar geleden,@farjadmir8842,UCOnwuwmQK7JFALIdHkLNhHw,41,https://yt3.ggpht.com/ytc/AMLnZu_V78-vhTNlmKWf...,1.0
2,Ugws4H_1H0bTzKyoDNR4AaABAg,Thank you for bringing up this amazing video s...,1 jaar geleden,@AlonAvramson,UCCAhR_5ynFFnGt18le759gA,8,https://yt3.ggpht.com/ytc/AMLnZu9IcQywbzPzmbeK...,0.0
3,UgxnaWdHpn_qWhPY9Kh4AaABAg,Sir I am humbled by your attitude. Your compre...,1 jaar geleden,@siddharthvm8262,UCTm70_j2-6F-kKjP1l5nnuQ,4,https://yt3.ggpht.com/ytc/AMLnZu9cqz_AG7msagpK...,0.0
4,UgwKZ6hO890gv2oCrvd4AaABAg,Time to take Tensorflow 2.0 outta my bucketlis...,1 jaar geleden,@dec13666,UCnrdL6SiCIM-K_TETJt05GA,0,https://yt3.ggpht.com/ytc/AMLnZu9XWJfpqCuxfvrx...,0.0
5,UgzM2SzHpQhqvcEmYlh4AaABAg,Hi sir... I m new to your channel. And i reall...,2 jaar geleden,@rahulagarwal8059,UCUjekYBlVyxnO1UW7dEi6NA,0,https://yt3.ggpht.com/ytc/AMLnZu80D07O-ByrzUaG...,0.0
6,UgwpjypihK6_PEtJF8h4AaABAg,thank u so so so much sir.. i am waiting for t...,2 jaar geleden,@omarf2741,UCh8Q-pNPFUBvOMrjYAZKFgg,2,https://yt3.ggpht.com/ytc/AMLnZu85WRyW4aIKXFIk...,0.0
7,Ugw6awZ5JAWCxMBrbch4AaABAg,"sir, sir , sir, sir .....\ni am extremely exci...",2 jaar geleden,@abdultaufiq2237,UCGRo5wmzwlSK3P6istFgrug,41,https://yt3.ggpht.com/ytc/AMLnZu9bfzO1fQfaky2F...,0.0
8,UgzZwMn9f1H69CNunkN4AaABAg,Thank you one again dhaval to making a video s...,2 jaar geleden,@dholanaresh2584,UCpzRFCffqsMjXe43xtxzQbQ,4,https://yt3.ggpht.com/ytc/AMLnZu9qOPhOeweVjfn9...,1.0
9,UgzX45_QVvLJtePfyDJ4AaABAg,exercises really help sir...they reinforces th...,1 jaar geleden,@subhamsekharpradhan297,UCcZFRmW0BmEbnfoIZ8Hj8hQ,0,https://yt3.ggpht.com/ytc/AMLnZu9TknX6Fa36YN7a...,0.0



[0.87 seconds] Done!
DataFrame Shape:  (20, 8) 
Comment DataFrame: 


,cid,text,time,author,channel,votes,photo,heart
0,UgwCXreyUx-gpEqkT194AaABAg,Please make sure you subscribe my vlogging cha...,1 jaar geleden,Krish Naik,UCNU_lfiiWBdtULKOw6X0Dig,8,https://yt3.ggpht.com/ytc/AMLnZu_Ow7V1PyNJXmvx...,0.0
1,UgyVWIPSyTicfrdkd514AaABAg,"Krish sir, your community has very huge knowle...",1 jaar geleden,Narsimha Rao,UCya_tJNfWRUBvYpUJo5AF9A,13,https://yt3.ggpht.com/ytc/AMLnZu8xAthuQ9lx0xpn...,1.0
2,Ugzq5Ljz3zKdmUoY5Cl4AaABAg,I've also shifted myself to deep learning algo...,1 jaar geleden,Azaz Butt,UCySlq27K588-DPw_aLSmmwA,11,https://yt3.ggpht.com/ytc/AMLnZu8sHVbPxYu_Ke7B...,0.0
3,UgxfVSdBqTQpSshcLnR4AaABAg,"HI Mr.Krish Naik, I keep on watching your vide...",1 jaar geleden,Bala Subramaniyan,UCPmzlqZB97LqqKijG1FFpTA,0,https://yt3.ggpht.com/ytc/AMLnZu8cvVpq07Gw4RHv...,0.0
4,UgwmhxHxXoUlQyITFDl4AaABAg,Hey I have a question regarding encoding my ‘C...,1 jaar geleden (bewerkt),Omer Ahmed,UCqCMvWwN5hvycyBNSQWLfEw,1,https://yt3.ggpht.com/ytc/AMLnZu92HoLu3ehBCuZs...,0.0
5,UgwNEEth1Bj5njL44SV4AaABAg,Sir when can we expect you to upload deeplearn...,1 jaar geleden,Abhishek_DataMan,UCSw7D4YiVeEJTR2quTV-_ew,8,https://yt3.ggpht.com/wbUx2_JwRgZw7h25tXTEqh_K...,0.0
6,UgxLH_pKMo5hEa8RUpR4AaABAg,one of the best playlists for deep learning,4 maanden geleden,Tanvir Ahmed,UCEjZvH0AOMuieWghLGUjg9Q,0,https://yt3.ggpht.com/ytc/AMLnZu8HUwO9D_SHkqvF...,0.0
7,UgyijxzKeTaXGkjOryh4AaABAg,Very helpful video .. Thank you Sir !,1 jaar geleden,Uddalak Mitra,UCYHbOIVS_M97IVJycvOJFEg,0,https://yt3.ggpht.com/ytc/AMLnZu-EZ3JiGYg8g07z...,0.0
8,UgwmtZA_OVihyiPFDuN4AaABAg,Krish sir plss make a video on transformer and...,1 jaar geleden,raavan hun main,UCW7w3amE9_cN92LkN-B4awg,2,https://yt3.ggpht.com/ytc/AMLnZu-syMU0Z9hD7NHv...,0.0
9,UgyNNB003P3rREac4XJ4AaABAg,"Krish sir, can you create a vedio on GAN models?",1 jaar geleden,PPKumar,UC9M_hRPtuLZ9I0MUBKrRfmQ,2,https://yt3.ggpht.com/I09utJV5KmG48vJ1tZgCnal8...,0.0



[0.98 seconds] Done!
DataFrame Shape:  (20, 8) 
Comment DataFrame: 


,cid,text,time,author,channel,votes,photo,heart
0,UgyKqgDZbWihkGIleaF4AaABAg,What else could one ask as a weekend treat?!,9 maanden geleden,Ajay Taneja,UCuUT8n_X7UI0H746IWl9Wkw,116,https://yt3.ggpht.com/ytc/AMLnZu9EdDZ8SKYp1pnK...,1.0
1,UgzJ-z29lb5_kgJQiO14AaABAg,Super Excited to learn! Thank you MIT folks fo...,9 maanden geleden,Ashish Kumar Choubey,UCePL2x3UhtrtBm5U3Kb8IBg,163,https://yt3.ggpht.com/ytc/AMLnZu8wU9yJ2bSrBVVW...,1.0
2,Ugz9TYCSm0FoX7pwW3F4AaABAg,Alexander is a master in presenting super comp...,8 maanden geleden,sarthak mohanty,UC6DXrhIMl6ptrw-n0gixPlQ,15,https://yt3.ggpht.com/ytc/AMLnZu_IqkxQu3u0OKQG...,1.0
3,UgwcYYpcbFVOH4MWyv14AaABAg,Just watched the lecture and I'm amazed at how...,8 maanden geleden,Vlad-Emil Flore,UCBHSVaIxrDXCKDyXoBo38Ig,30,https://yt3.ggpht.com/ytc/AMLnZu926iuDodVQWzZ5...,1.0
4,UgxKviaTkfGy_eo2B_54AaABAg,I’m citing you in my high school project! Than...,8 maanden geleden,Scrap!,UC8zRG2e2bHo52mHJvuraoUA,21,https://yt3.ggpht.com/ytc/AMLnZu89OeteEXTDZlXc...,1.0
5,UgzzaldZivDAyuhcStx4AaABAg,Thank you Professor! Its really great to watch...,8 maanden geleden,Saurabh Mishra,UCq1p6zwCA3K-VzJyE5N88kA,3,https://yt3.ggpht.com/ytc/AMLnZu96NXTAmIRxIRvh...,1.0
6,UgxiGTxQxDYX4O0LhCN4AaABAg,Year number 3 now that I am following 6.S191....,8 maanden geleden,anand khandekar,UCTP1wp-MQU5f3eyp67Rnb7Q,2,https://yt3.ggpht.com/ytc/AMLnZu8AG0ebaKtRedKI...,1.0
7,UgytGVNx9UQgnPhCCnx4AaABAg,This is unbelievable 🔥 how good could a course...,8 maanden geleden,Thomas,UC4nhT8HrdcqdLmwv9K5zNDw,3,https://yt3.ggpht.com/hoOHoFVZF9MASz5Ir802E7xB...,1.0
8,UgzgA5TV7glujcQtphB4AaABAg,Working with deep learning on my master thesis...,8 maanden geleden,CHITUS💖⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻⸻,UCzw86ClXJvK97Nw1BDe2-BQ,3,https://yt3.ggpht.com/wOaYb3tmddwDd4PQZgjsOZ-D...,0.0
9,UgybEfTs-p8f-Bdem014AaABAg,This was really really helpful. Thank you MIT ...,6 maanden geleden,PTE tips - සිo හ ලෙ න්,UCPHe4RiuwRE7RtPdRX7jW4w,2,https://yt3.ggpht.com/ytc/AMLnZu8Qa1ljhXpVzKR7...,1.0



[0.83 seconds] Done!


## Output 

In [23]:
df_comment = pd.read_csv('./ytb_comments.csv')

df_comment.head()

,cid,text,time,author,channel,votes,photo,heart
0,UgyQFWUAwflZw8dgkUl4AaABAg,Shot horizontally😀👍,1 jaar geleden,Dave Ellis,UC2vxO8ZCkPanvHF5L4IQ79Q,340,https://yt3.ggpht.com/ytc/AMLnZu_80GFCyeIQRDI_...,1.0
1,UgwtAC15ZpDsqRFscuB4AaABAg,I have a feeling we aren't too far from Disney...,11 maanden geleden,Stefan,UCn_cd_kyCntJs4yu249FayQ,259,https://yt3.ggpht.com/ytc/AMLnZu9dxMjVLZVsbgtJ...,1.0
2,Ugxl-2CmqdAsscaV9FJ4AaABAg,That’s incredible. I can’t believe how much wo...,11 maanden geleden,Tim Moeller,UCLjV7s8SwS3STCvWCZ_Znqw,177,https://yt3.ggpht.com/qA5LGkIhqbY3mRa0PEM01uui...,1.0
3,Ugxjl5CzAJ928JZ6pdt4AaABAg,I love stuff like this because I feel like art...,1 jaar geleden,Jack Middleton,UCHNIRmyVDh52gm60BDEzyCw,126,https://yt3.ggpht.com/ytc/AMLnZu92mQwpJfkOX2Qh...,1.0
4,UgxSGTgzQNP0B0e7OPh4AaABAg,That…was…EPIC! Seeing that live would have bee...,11 maanden geleden,Isaac Sheppard,UC-Kkc303MEP_1heObyPye8Q,78,https://yt3.ggpht.com/ytc/AMLnZu8nVB2_P5WkAx-_...,1.0


In [24]:
df_comment.describe()

,heart
count,372.000000
mean,0.231183
std,0.422157
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [27]:
df_comment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cid      372 non-null    object 
 1   text     372 non-null    object 
 2   time     372 non-null    object 
 3   author   372 non-null    object 
 4   channel  372 non-null    object 
 5   votes    372 non-null    object 
 6   photo    372 non-null    object 
 7   heart    372 non-null    float64
dtypes: float64(1), object(7)
memory usage: 23.4+ KB


In [31]:
df_comment.text.tail(10)

362    Never thought that someday I would be able to ...
363    I have to say, this is one of the best classes...
364    Thanks for providing these great lectures! Are...
365    ​Hi Alexander! I've been fan of you and Ava si...
366    Good work Alexander. Keep it up. I'll be watch...
367    An excellent introduction to deep learning. Cr...
368    Thanks for making such an awesome series of le...
369    Hi , Alexander , you just explained deep learn...
370    I also teach deep learning and when I see clas...
371    A-MA-ZING. Looking forward to the rest of the ...
Name: text, dtype: object

In [33]:
df_comment.info

<bound method DataFrame.info of                             cid                                               text                          time                                             author                   channel votes                                              photo  heart
0    UgyQFWUAwflZw8dgkUl4AaABAg                                Shot horizontally😀👍                1 jaar geleden                                         Dave Ellis  UC2vxO8ZCkPanvHF5L4IQ79Q   340  https://yt3.ggpht.com/ytc/AMLnZu_80GFCyeIQRDI_...    1.0
1    UgwtAC15ZpDsqRFscuB4AaABAg  I have a feeling we aren't too far from Disney...            11 maanden geleden                                             Stefan  UCn_cd_kyCntJs4yu249FayQ   259  https://yt3.ggpht.com/ytc/AMLnZu9dxMjVLZVsbgtJ...    1.0
2    Ugxl-2CmqdAsscaV9FJ4AaABAg  That’s incredible. I can’t believe how much wo...            11 maanden geleden                                        Tim Moeller  UCLjV7s8SwS3STCvWCZ_Znqw   177  https://y

## Reference

1. [https://github.com/egbertbouman/youtube-comment-downloader](https://github.com/egbertbouman/youtube-comment-downloader)
2. https://www.kaggle.com/code/ahmedshahriarsakib/scrape-youtube-comments-for-free-no-google-api